# Crear JSONs

In [22]:
import pandas as pd
import os

# 1. Configuración de salida
os.makedirs('../data/processed', exist_ok=True)

def procesar_ano(df, ano):
    # Diccionario de mapeo basado en tus listas
    # Nota: He añadido 'ANYO' porque así viene en tus archivos
    mapeo = {
            'ANYO': 'ANIO',
            'COD_PROVINCIA': 'PROVINCIA',
            'TIPO_VIA': 'TIPO_VIA_NOMBRE',
            'TIPO_ACCIDENTE': 'TIPO_ACCIDENTE_NOMBRE',
            'CONDICION_METEO': 'CONDICION_METEO', 
            'CARRETERA': 'CARRETERA'
        }
    df = df.rename(columns=mapeo)
    
    # 2. Rellenar nulos para las nuevas columnas de texto (Evita errores en el modelo)
    for col in ['CONDICION_METEO', 'CARRETERA', 'DIA_SEMANA', 'CONDICION_ILUMINACION']:
        if col in df.columns:
            df[col] = df[col].fillna("DESCONOCIDO")
    # Columnas de fallecidos que me pasaste (exactas)
    fallecidos_cols = [
        'TOT_PEAT_MU24H', 'TOT_BICI_MU24H', 'TOT_CICLO_MU24H', 'TOT_MOTO_MU24H',
        'TOT_TUR_MU24H', 'TOT_FURG_MU24H', 'TOT_CAM_MENOS3500_MU24H', 
        'TOT_CAM_MAS3500_MU24H', 'TOTAL_MU24H'
    ]

    # Asegurar que existan y no tengan nulos
    for col in fallecidos_cols:
        if col not in df.columns:
            df[col] = 0
        df[col] = df[col].fillna(0)

    # Lógica de unificación para las gráficas
    df['MUERTOS_MOTO'] = df['TOT_CICLO_MU24H'] + df['TOT_MOTO_MU24H']
    df['MUERTOS_COCHE'] = df['TOT_TUR_MU24H']
    df['MUERTOS_BICI'] = df['TOT_BICI_MU24H']
    
    # Variable objetivo para el modelo y gráficas
    df['ES_MORTAL'] = (df['TOTAL_MU24H'] > 0).astype(int)
    df['ANIO'] = ano # Forzamos el año del archivo

    # Selección final de columnas para los JSON
    cols_web = [
        'ANIO', 'HORA', 'MES', 'DIA_SEMANA', 'PROVINCIA', 
        'TIPO_VIA_NOMBRE', 'TIPO_ACCIDENTE_NOMBRE', 
        'CONDICION_METEO', 'CARRETERA', 'CONDICION_ILUMINACION',
        'MUERTOS_MOTO', 'MUERTOS_COCHE', 'MUERTOS_BICI', 
        'ES_MORTAL', 'TOTAL_MU24H' # Añadimos TOTAL_MU24H para el JSON de carreteras
    ]
    
    # Solo devolvemos las que existan para evitar KeyErrors
    existentes = [c for c in cols_web if c in df.columns]
    return df[existentes]

# --- CARGA Y UNIFICACIÓN ---

archivos = {
    2020: '../data/raw/acc_2020.xlsx', 
    2021: '../data/raw/acc_2021.xlsx', 
    2022: '../data/raw/acc_2022.xlsx', 
    2023: '../data/raw/acc_2023.xlsx',
    2024: '../data/raw/acc_2024.xlsx'
}

datasets = []
for ano, path in archivos.items():
    print(f"⌛ Procesando {ano}...")
    temp_df = pd.read_excel(path, engine='openpyxl')
    datasets.append(procesar_ano(temp_df, ano))

df_historico = pd.concat(datasets, ignore_index=True)

# --- EXPORTACIÓN JSON ---

# 1. Evolución (Barras)
df_historico.groupby('ANIO').agg({
    'MUERTOS_MOTO': 'sum', 'MUERTOS_COCHE': 'sum', 
    'MUERTOS_BICI': 'sum', 'ES_MORTAL': 'count'
}).reset_index().to_json('../data/processed/stats_evolucion.json', orient='records')

# 2. Hora (Línea)
df_historico.groupby('HORA')['ES_MORTAL'].mean().reset_index().to_json('../data/processed/prob_mortalidad_hora.json', orient='records')

print("✅ Archivos JSON generados correctamente.")

⌛ Procesando 2020...
⌛ Procesando 2021...
⌛ Procesando 2022...
⌛ Procesando 2023...
⌛ Procesando 2024...
✅ Archivos JSON generados correctamente.


In [23]:
# 3. Riesgo por Meteorología
df_historico.groupby('CONDICION_METEO').agg(
    Total_Accidentes=('ES_MORTAL', 'count'),
    Probabilidad_Mortal=('ES_MORTAL', 'mean')
).reset_index().to_json('../data/processed/stats_riesgo_meteo.json', orient='records')

# 4. Top 20 Carreteras (Puntos Negros)
df_historico.groupby('CARRETERA').agg(
    Total_Accidentes=('ES_MORTAL', 'count'),
    Total_Muertos=('TOTAL_MU24H', 'sum'),
    Riesgo_Tramo=('ES_MORTAL', 'mean')
).reset_index().sort_values(by='Total_Accidentes', ascending=False).head(20).to_json('../data/processed/stats_riesgo_carreteras.json', orient='records')

# 5. Tipología de Accidente de Moto
df_historico.groupby('TIPO_ACCIDENTE_NOMBRE').agg(
    Frecuencia=('ES_MORTAL', 'count'),
    Mortalidad_Media=('ES_MORTAL', 'mean')
).reset_index().to_json('../data/processed/stats_motos_tipologia.json', orient='records')

print("🚀 Todos los JSONs estratégicos han sido generados.")

🚀 Todos los JSONs estratégicos han sido generados.


In [24]:
# 1. Filtramos accidentes donde hubo al menos un motorista fallecido o implicado
# (Usamos MUERTOS_MOTO > 0 para ver letalidad específica)
df_motos = df_historico[df_historico['MUERTOS_MOTO'] > 0].copy()

# 2. Agrupamos por tipo de accidente
stats_motos = df_historico.groupby('TIPO_ACCIDENTE_NOMBRE').agg(
    Frecuencia=('ES_MORTAL', 'count'),      # Cuántos accidentes de este tipo hay
    Tasa_Mortalidad=('ES_MORTAL', 'mean')   # Probabilidad de que sea mortal
).reset_index()

# Filtramos tipos con muy pocos casos para que la gráfica no sea un caos
stats_motos = stats_motos[stats_motos['Frecuencia'] > 20]

stats_motos.to_json('../data/processed/stats_motos_tipologia.json', orient='records')

In [25]:
stats_provincias = df_historico.groupby('PROVINCIA').agg(
    Total_Accidentes=('ES_MORTAL', 'count'),
    Muertos_Moto=('MUERTOS_MOTO', 'sum'),
    Riesgo_Medio=('ES_MORTAL', 'mean')
).reset_index()

stats_provincias.to_json('../data/processed/stats_provincias.json', orient='records')

In [26]:
stats_meses = df_historico.groupby('MES').agg(
    Muertos_Moto=('MUERTOS_MOTO', 'sum'),
    Muertos_Coche=('MUERTOS_COCHE', 'sum')
).reset_index()

stats_meses.to_json('../data/processed/stats_meses.json', orient='records')